In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import vincent

## Data Cleaning 

In [2]:
countries = ['Switzerland','United States', 'India','France', 'Italy','Australia','Singapore', 'Japan','Kenya','Brazil']

In [3]:
data = pd.DataFrame()

for file in (os.listdir('raw_data/')[1:]):    
    df = pd.read_excel('raw_data/'+ file, header=3)
    df.set_index('Country Name', inplace=True)
    df = df.loc[countries]
    df = df[df.columns[0:2].append(df.columns[32:57])]
    df.drop('Country Code', axis = 1, inplace=True)
    data = data.append(df)
    
data.sort_index(inplace=True)
data.reset_index(inplace=True)
data.set_index( ['Country Name', 'Indicator Name'],inplace=True)

for country in countries:
    data.loc[country].transpose().to_csv('data_countries/'+country + '.csv')

In [4]:
for country in countries:
    con_data = pd.read_csv('data_countries/'+country +'.csv')
    con_data.rename( columns={'Unnamed: 0':'Year'}, inplace=True )
    con_data['Trade in services (% of GDP)'] = con_data['Trade in services (% of GDP)']*con_data['GDP at market prices (constant 2010 US$)']/100
    con_data['Trade (% of GDP)'] = con_data['Trade (% of GDP)']*con_data['GDP at market prices (constant 2010 US$)']/100
    con_data['Gross domestic savings (% of GDP)'] = con_data['Gross domestic savings (% of GDP)']*con_data['GDP at market prices (constant 2010 US$)']/100
    con_data.set_index('Year', inplace=True)
    con_data.rename( columns={'Gross domestic savings (% of GDP)': 'Gross domestic savings (constant 2010 US$)','Trade (% of GDP)':'Trade (constant 2010 US$)','Trade in services (% of GDP)':'Trade in services (constant 2010 US$)'}, inplace=True )
    con_data[['Household final consumption expenditure per capita (% change)','Trade in services (% change)','Electric power consumption (% change)','Foreign direct investment, net inflows (% change)','Gross domestic savings (% change)','GNI per capita (% change)','Cost to import (% change)','Trade (% change)','GDP per capita (% change)']]=con_data[['Household final consumption expenditure per capita (constant 2010 US$)','Trade in services (constant 2010 US$)','Electric power consumption (kWh per capita)','Foreign direct investment, net inflows (BoP, current US$)','Gross domestic savings (constant 2010 US$)','GNI per capita (constant 2010 US$)','Cost to import (US$ per container)','Trade (constant 2010 US$)','GDP per capita (constant 2010 US$)']].pct_change()*100 
    con_data.drop(['GDP at market prices (constant 2010 US$)'], axis=1, inplace=True)
    con_data['Inflation, consumer prices (annual %)']=con_data['Inflation, consumer prices (annual %)'].shift(+1)
    con_data.drop( 1989, inplace = True)
    con_data = con_data.apply(lambda x: round(x,2), axis=1) 
    con_data.to_csv('data_countries/'+country+'.csv')

In [5]:
full_df = pd.DataFrame()
for country in countries:
    con_df = pd.read_csv('data_countries/'+country+'.csv')
    con_df['Country'] = country
    full_df = full_df.append(con_df)
full_df.set_index(['Country','Year'], inplace=True)
full_df.sort_index(inplace=True)
full_df.to_csv('data_countries/0_Data_All_Countries.csv')

In [6]:
full_df.columns

Index(['Cost to import (% change)', 'Cost to import (US$ per container)',
       'Deposit interest rate (%)', 'Electric power consumption (% change)',
       'Electric power consumption (kWh per capita)',
       'Foreign direct investment, net inflows (% change)',
       'Foreign direct investment, net inflows (BoP, current US$)',
       'GDP per capita (% change)', 'GDP per capita (constant 2010 US$)',
       'GNI per capita (% change)', 'GNI per capita (constant 2010 US$)',
       'Gross domestic savings (% change)',
       'Gross domestic savings (constant 2010 US$)',
       'Household final consumption expenditure per capita (% change)',
       'Household final consumption expenditure per capita (constant 2010 US$)',
       'Inflation, consumer prices (annual %)', 'Lending interest rate (%)',
       'Life expectancy at birth, total (years)',
       'Merchandise imports from high-income economies (% of total merchandise imports)',
       'Trade (% change)', 'Trade (constant 2010 US$

In [7]:
full_df.loc['Switzerland']['Inflation, consumer prices (annual %)'].corr(full_df.loc['Switzerland']['Household final consumption expenditure per capita (% change)'])

-0.49192566792491527

In [8]:
full_df.loc['United States']['Inflation, consumer prices (annual %)'].corr(full_df.loc['United States']['Household final consumption expenditure per capita (% change)'])

-0.3864926628058557

In [9]:
full_df.loc['India']['Inflation, consumer prices (annual %)'].corr(full_df.loc['India']['Household final consumption expenditure per capita (% change)'])

0.022604440236969666

In [10]:
full_df.loc['Australia']['Inflation, consumer prices (annual %)'].corr(full_df.loc['Australia']['Household final consumption expenditure per capita (% change)'])

-0.30108446661214938

In [11]:
full_df.loc['France']['Inflation, consumer prices (annual %)'].corr(full_df.loc['France']['Household final consumption expenditure per capita (% change)'])

-0.42224554021790467

In [12]:
full_df.loc['Italy']['Inflation, consumer prices (annual %)'].corr(full_df.loc['Italy']['Household final consumption expenditure per capita (% change)'])

0.12747205455931818

In [13]:
full_df.loc['Singapore']['Inflation, consumer prices (annual %)'].corr(full_df.loc['Singapore']['Household final consumption expenditure per capita (% change)'])

-0.45939906617539256

In [14]:
full_df.loc['Japan']['Inflation, consumer prices (annual %)'].corr(full_df.loc['Japan']['Household final consumption expenditure per capita (% change)'])

0.15455903329104417

In [15]:
full_df.loc['Kenya']['Inflation, consumer prices (annual %)'].corr(full_df.loc['Kenya']['Household final consumption expenditure per capita (% change)'])

-0.017945410253291546

In [16]:
full_df.loc['Brazil']['Inflation, consumer prices (annual %)'].corr(full_df.loc['Brazil']['Household final consumption expenditure per capita (% change)'])

0.46659529477574857

# PLOTTING

In [2]:
full_df = pd.read_csv('data_countries/0_Data_All_Countries.csv', index_col=['Country', 'Year'])

In [3]:
vincent.core.initialize_notebook()

In [4]:
for country in ['Australia', 'India', 'Switzerland', 'France']:
    grouped = vincent.GroupedBar(full_df.loc[country][['Household final consumption expenditure per capita (% change)','Inflation, consumer prices (annual %)','GDP per capita (% change)']])
    grouped.width = 700
    grouped.height = 170
    grouped.colors(brew='Pastel1')
    grouped.axis_titles(x='year', y='Percentage',)
    grouped.legend(title='Index')
    grouped.name = country
    grouped.display()
